motive: We are attempting to use a constant-scaling method to perform endpoint estimations for beta and (possibly) gamma distributions. 
Basic steps are as follows:
1. Simulate a bunch of points and take some upper proportion p (upper thirds, upper quartile, etc.)
2. Calculate the standard deviation of the sample
3. Figure out what constant $c * \hat{\sigma}$ added to the sample maximum would get close to 1
4. Repeat 1-3 and obtain average of $c$ s.
5. Test on different Beta distributions (and maybe Gamma distributions). Is there a general rule?

In [7]:
#Import libraries here
#numpy help me
import numpy as np 
seed = 2024

In [9]:
#Start with Beta(2,5) distribution
rng = np.random.default_rng(seed=seed)
sample = rng.beta(2,5,1000) #sample 1000 points from distribution
cutoff = np.percentile(sample, 66.7) #find cutoff for top third
test_sample = sample[sample > cutoff] #get top third
print(test_sample)

[0.45970148 0.46435208 0.49293456 0.4224627  0.49163051 0.4975371
 0.36015092 0.41969849 0.42432284 0.55822839 0.40641933 0.3662103
 0.55325939 0.54946863 0.5035101  0.54556018 0.39860529 0.59275488
 0.50394437 0.4119537  0.4200089  0.4661882  0.78625588 0.39497863
 0.54618236 0.39235106 0.73605842 0.47941161 0.44912143 0.46238015
 0.56153255 0.37217264 0.4853603  0.46200572 0.44195374 0.4803264
 0.60868519 0.40949599 0.55815736 0.86059883 0.53012512 0.54694422
 0.58408626 0.45727862 0.47544664 0.38339356 0.58865519 0.43769816
 0.37340053 0.47662678 0.54365185 0.44972509 0.59325537 0.39163823
 0.44170769 0.42786578 0.42141725 0.46818805 0.38976054 0.48415251
 0.47708368 0.39801484 0.5652027  0.39135454 0.45783885 0.46464726
 0.54046646 0.39735615 0.36875114 0.41789919 0.57207026 0.45854585
 0.59235754 0.36570543 0.45492396 0.52604239 0.4821784  0.50065184
 0.52194396 0.55755361 0.44676696 0.38318522 0.51588403 0.4679654
 0.48995219 0.45967128 0.39275619 0.61305716 0.45682303 0.45673571